In [1]:
import pandas as pd
df=pd.read_csv("Salman_Khan_VS_Sharuk_Khan - Sheet2.csv")
df.head()


,Name,Comment,Time,Likes,Reply Count,Supporter
0,Obed Khan,Shah Rukh Khan sirf ek,2023-01-30T02:44:56Z,0,0,Shahrukh khan
1,Aasif Patel,Salman khan,2023-01-29T13:49:48Z,0,0,Salman Khan
2,V Shejule Status,Only Salman bhai<br>Bhot Hard Bhai✌,2023-01-28T17:53:55Z,0,0,Salman Khan
3,Avinash Tudu rushika,Bollywood ka sabse bada super star srk,2023-01-28T17:38:39Z,0,0,Shahrukh khan
4,DexterDrubo Animations-,In India Salman Khan has bigger stardom than S...,2023-01-28T11:34:45Z,0,0,Shahrukh khan


In [2]:
df['Supporter']=df['Supporter'].str.lower()
df['Supporter'].unique()

array(['shahrukh  khan', 'salman khan', 'both', 'comment', 'sharukh khan',
       'shahrukh khan', ' shahrukh khan', nan, 'comments', 'ssha',
       'cooment', 'ssa', 'commets'], dtype=object)

In [3]:
df['Supporter'].value_counts()

salman khan       461
sharukh khan      386
comment            62
shahrukh  khan     61
both               20
shahrukh khan       2
comments            2
 shahrukh khan      1
ssha                1
cooment             1
ssa                 1
commets             1
Name: Supporter, dtype: int64

In [4]:
ind1=df[df['Supporter'].str.contains("sharukh khan", na=False)].index
ind2=df[df['Supporter'].str.contains("shahrukh  khan", na=False)].index
ind3=df[df['Supporter'].str.contains(" shahrukh khan", na=False)].index


In [5]:
df.loc[ind1,'Supporter']="shahrukh khan" 
df.loc[ind2,'Supporter']="shahrukh khan"
df.loc[ind3,'Supporter']="shahrukh khan" 

In [6]:
df['Supporter'].unique()

array(['shahrukh khan', 'salman khan', 'both', 'comment', nan, 'comments',
       'ssha', 'cooment', 'ssa', 'commets'], dtype=object)

In [7]:
search=["shahrukh khan","salman khan"]
drop_ind=df[~df['Supporter'].str.contains('|'.join(search), na=False)].index

In [8]:
df.drop(drop_ind, inplace=True)



In [9]:
df['Supporter'].unique()

array(['shahrukh khan', 'salman khan'], dtype=object)

In [10]:
# Parameters

vocab_size = 100000
max_length = 120
embedding_dim = 16
trunc_type='post'
oov_tok = "<OOV>"

In [11]:
import numpy as np
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
Supporter=df['Supporter']
Supporter=np.array(Supporter).reshape(-1,1)


df['encoded_labels']=''
enc=le.fit_transform(Supporter)

df['encoded_labels']=enc
df.head()

C:\Users\DELL\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


,Unnamed: 0,Name,Comment,Time,Likes,Reply Count,Supporter,encoded_labels
0,0,Obed Khan,Shah Rukh Khan sirf ek,2023-01-30T02:44:56Z,0,0,shahrukh khan,1
1,1,Aasif Patel,Salman khan,2023-01-29T13:49:48Z,0,0,salman khan,0
2,2,V Shejule Status,Only Salman bhai<br>Bhot Hard Bhai✌,2023-01-28T17:53:55Z,0,0,salman khan,0
3,3,Avinash Tudu rushika,Bollywood ka sabse bada super star srk,2023-01-28T17:38:39Z,0,0,shahrukh khan,1
4,4,DexterDrubo Animations-,In India Salman Khan has bigger stardom than S...,2023-01-28T11:34:45Z,0,0,shahrukh khan,1


In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Initialize the Tokenizer class
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)

# Generate the word index dictionary for the training sentences
tokenizer.fit_on_texts(df["Comment"])
word_index = tokenizer.word_index

# Generate and pad the training sequences
sequences = tokenizer.texts_to_sequences(df["Comment"])
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)



In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded,df["encoded_labels"])

In [14]:
import tensorflow as tf

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, return_sequences=True)),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Setup the training parameters
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           160000    
                                                                 
 bidirectional (Bidirectiona  (None, 120, 512)         420864    
 l)                                                              
                                                                 
 global_average_pooling1d (G  (None, 512)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 64)                32832     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 6

In [15]:
model.fit(X_train,y_train, epochs=500,validation_data=(X_test, y_test ))

Epoch 1/500
22/22 [==============================] - 8s 286ms/step - loss: 0.6948 - accuracy: 0.4788 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/500
22/22 [==============================] - 6s 262ms/step - loss: 0.6944 - accuracy: 0.4890 - val_loss: 0.6930 - val_accuracy: 0.5000
Epoch 3/500
22/22 [==============================] - 6s 265ms/step - loss: 0.6948 - accuracy: 0.5154 - val_loss: 0.6930 - val_accuracy: 0.5000
Epoch 4/500
22/22 [==============================] - 6s 277ms/step - loss: 0.6925 - accuracy: 0.5168 - val_loss: 0.6926 - val_accuracy: 0.5000
Epoch 5/500
22/22 [==============================] - 8s 356ms/step - loss: 0.6917 - accuracy: 0.5212 - val_loss: 0.6907 - val_accuracy: 0.5439
Epoch 6/500
22/22 [==============================] - 7s 335ms/step - loss: 0.6290 - accuracy: 0.7028 - val_loss: 0.4062 - val_accuracy: 0.8509
Epoch 7/500
22/22 [==============================] - 7s 328ms/step - loss: 0.5043 - accuracy: 0.8023 - val_loss: 0.6202 - val_accuracy: 0.7149

Epoch 58/500
22/22 [==============================] - 9s 422ms/step - loss: 0.0819 - accuracy: 0.9824 - val_loss: 0.7799 - val_accuracy: 0.8202
Epoch 59/500
22/22 [==============================] - 9s 391ms/step - loss: 0.0793 - accuracy: 0.9824 - val_loss: 0.7318 - val_accuracy: 0.8114
Epoch 60/500
22/22 [==============================] - 8s 374ms/step - loss: 0.0777 - accuracy: 0.9839 - val_loss: 0.7765 - val_accuracy: 0.8114
Epoch 61/500
22/22 [==============================] - 8s 364ms/step - loss: 0.0793 - accuracy: 0.9839 - val_loss: 0.7595 - val_accuracy: 0.8114
Epoch 62/500
22/22 [==============================] - 8s 364ms/step - loss: 0.0780 - accuracy: 0.9839 - val_loss: 0.8162 - val_accuracy: 0.8114
Epoch 63/500
22/22 [==============================] - 8s 360ms/step - loss: 0.0774 - accuracy: 0.9839 - val_loss: 0.8084 - val_accuracy: 0.8114
Epoch 64/500
22/22 [==============================] - 8s 357ms/step - loss: 0.0835 - accuracy: 0.9810 - val_loss: 0.8044 - val_accuracy:

Epoch 115/500
22/22 [==============================] - 9s 388ms/step - loss: 0.0612 - accuracy: 0.9868 - val_loss: 0.9351 - val_accuracy: 0.8070
Epoch 116/500
22/22 [==============================] - 9s 386ms/step - loss: 0.0608 - accuracy: 0.9868 - val_loss: 0.9951 - val_accuracy: 0.8026
Epoch 117/500
22/22 [==============================] - 8s 382ms/step - loss: 0.0643 - accuracy: 0.9868 - val_loss: 1.0073 - val_accuracy: 0.8026
Epoch 118/500
22/22 [==============================] - 9s 386ms/step - loss: 0.0668 - accuracy: 0.9868 - val_loss: 1.0836 - val_accuracy: 0.8070
Epoch 119/500
22/22 [==============================] - 8s 382ms/step - loss: 0.0694 - accuracy: 0.9868 - val_loss: 0.8981 - val_accuracy: 0.8026
Epoch 120/500
22/22 [==============================] - 9s 390ms/step - loss: 0.0678 - accuracy: 0.9868 - val_loss: 0.9765 - val_accuracy: 0.7982
Epoch 121/500
22/22 [==============================] - 9s 402ms/step - loss: 0.0625 - accuracy: 0.9868 - val_loss: 1.0277 - val_ac

22/22 [==============================] - 9s 414ms/step - loss: 0.0372 - accuracy: 0.9898 - val_loss: 1.7862 - val_accuracy: 0.7807
Epoch 228/500
22/22 [==============================] - 9s 423ms/step - loss: 0.0373 - accuracy: 0.9912 - val_loss: 1.8000 - val_accuracy: 0.7763
Epoch 229/500
22/22 [==============================] - 9s 417ms/step - loss: 0.0330 - accuracy: 0.9912 - val_loss: 1.8794 - val_accuracy: 0.7807
Epoch 230/500
22/22 [==============================] - 9s 408ms/step - loss: 0.0398 - accuracy: 0.9898 - val_loss: 1.8237 - val_accuracy: 0.7719
Epoch 231/500
22/22 [==============================] - 9s 391ms/step - loss: 0.0423 - accuracy: 0.9912 - val_loss: 1.8416 - val_accuracy: 0.7763
Epoch 232/500
22/22 [==============================] - 9s 402ms/step - loss: 0.0371 - accuracy: 0.9912 - val_loss: 1.8666 - val_accuracy: 0.7763
Epoch 233/500
22/22 [==============================] - 9s 401ms/step - loss: 0.0325 - accuracy: 0.9912 - val_loss: 1.9205 - val_accuracy: 0.7851

22/22 [==============================] - 9s 426ms/step - loss: 0.0378 - accuracy: 0.9898 - val_loss: 1.6627 - val_accuracy: 0.7807
Epoch 340/500
22/22 [==============================] - 10s 449ms/step - loss: 0.0360 - accuracy: 0.9927 - val_loss: 1.6621 - val_accuracy: 0.7851
Epoch 341/500
22/22 [==============================] - 9s 428ms/step - loss: 0.0324 - accuracy: 0.9912 - val_loss: 1.6860 - val_accuracy: 0.7851
Epoch 342/500
22/22 [==============================] - 9s 422ms/step - loss: 0.0344 - accuracy: 0.9912 - val_loss: 1.6906 - val_accuracy: 0.7807
Epoch 343/500
22/22 [==============================] - 9s 430ms/step - loss: 0.0356 - accuracy: 0.9912 - val_loss: 1.6870 - val_accuracy: 0.7807
Epoch 344/500
22/22 [==============================] - 10s 436ms/step - loss: 0.0327 - accuracy: 0.9912 - val_loss: 1.7213 - val_accuracy: 0.7851
Epoch 345/500
22/22 [==============================] - 10s 445ms/step - loss: 0.0354 - accuracy: 0.9912 - val_loss: 1.7766 - val_accuracy: 0.7

22/22 [==============================] - 9s 427ms/step - loss: 0.0331 - accuracy: 0.9912 - val_loss: 2.3138 - val_accuracy: 0.7675
Epoch 452/500
22/22 [==============================] - 10s 442ms/step - loss: 0.0325 - accuracy: 0.9912 - val_loss: 2.1651 - val_accuracy: 0.7719
Epoch 453/500
22/22 [==============================] - 10s 435ms/step - loss: 0.0405 - accuracy: 0.9912 - val_loss: 2.3090 - val_accuracy: 0.7675
Epoch 454/500
22/22 [==============================] - 10s 457ms/step - loss: 0.0381 - accuracy: 0.9912 - val_loss: 2.3063 - val_accuracy: 0.7675
Epoch 455/500
22/22 [==============================] - 10s 454ms/step - loss: 0.0346 - accuracy: 0.9912 - val_loss: 2.2400 - val_accuracy: 0.7763
Epoch 456/500
22/22 [==============================] - 10s 468ms/step - loss: 0.0335 - accuracy: 0.9912 - val_loss: 2.2554 - val_accuracy: 0.7675
Epoch 457/500
22/22 [==============================] - 9s 431ms/step - loss: 0.0356 - accuracy: 0.9912 - val_loss: 2.3427 - val_accuracy: 0

In [16]:
# The key difference between GRU and LSTM is that GRU's bag has two gates that are reset and update while
# LSTM has three gates that are input, output, forget. GRU is less complex than LSTM because it has less number of gates.
# If the dataset is small then GRU is preferred otherwise LSTM for the larger dataset.

In [16]:

prediction=model.predict(X_test)


8/8 [==============================] - 1s 45ms/step


In [17]:
Supporter=[]
for i in prediction:
    if i>=0.5:
        Supporter.append("Salman Khan")
    else:
        Supporter.append("Shahrukh Khan") 

In [18]:
Supporter[3:15]

['Shahrukh Khan',
 'Shahrukh Khan',
 'Salman Khan',
 'Salman Khan',
 'Shahrukh Khan',
 'Salman Khan',
 'Salman Khan',
 'Shahrukh Khan',
 'Shahrukh Khan',
 'Salman Khan',
 'Shahrukh Khan',
 'Salman Khan']

In [19]:
df["Comment"][3:15]

3                Bollywood ka sabse bada super star srk
4     In India Salman Khan has bigger stardom than S...
5                                                 Sallu
6                                           Srk is king
7                 Salman living in people&#39;s heart ❤
8                    Ladko ka favourite only sallu bhai
9            Salman bhai bus aur koi nhi hai un ke siva
10    Salman Khan ko hi King Khan hona hai srk bakwa...
11            Only bhaijan kyoki he has a pure heart 🥰🥰
12               Salman Khan is best hero for bollywood
13                                        Shahrukh Khan
14                          One n only Salman Khan bhai
Name: Comment, dtype: object

In [45]:
#Prediction
data=[input("Write your review about your favourite supporter\n")]
# Generate and pad the training sequences
sequencesp = tokenizer.texts_to_sequences(data)
paddedp = pad_sequences(sequencesp,maxlen=max_length, truncating=trunc_type)
prediction=model.predict(paddedp)
if prediction>0.5:
    print("Shahrukh Khan Supporter")
    
else:
    print("Salman Khan Supporter")

Write your review about your favourite supporter
King Khan
1/1 [==============================] - 0s 28ms/step
Shahrukh Khan Supporter
